# read file

In [2]:
DATA_DIR="data/ml-1m"
import pandas as pd
import numpy as np

In [6]:
data = pd.read_table("%s/ratings.dat"%DATA_DIR, delimiter="::", header=None)
data.rename(index=str, columns={0:'userID', 1:'itemID', 2:'ratings', 3:'timestamp'}, inplace=True)
data.head()

/Users/linh/PhD/code/.env3/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,userID,itemID,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
movies_info = pd.read_table("data/ml-1m/movies.dat", delimiter="::", header=None)
movies_info.rename(index=str, columns={0:'userID', 1:'itemID', 2:'ratings', 3:'timestamp'}, inplace=True)
movies_info.head()

/Users/linh/PhD/code/.env3/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# Matching new userID, itemID

In [3]:
item = sorted(data.itemID.unique())
user = sorted(data.userID.unique())
print("# num of user: %d \n# num of item: %d"%(len(user), len(item)))

# num of user: 6040 
# num of item: 3706


In [4]:
# Group data following user

n_user = len(user)
n_item = len(item)

previous_user_id = 0
ratings = []
for _, r in data.iterrows():
    if r.userID == previous_user_id:
        iid = item.index(r.itemID)
        ratings[-1].append([iid, r.ratings, r.timestamp])
    elif r.userID == previous_user_id + 1:
        iid = item.index(r.itemID)
        ratings.append([[iid, r.ratings, r.timestamp]])
        previous_user_id = r.userID
    else:
        print(r, previous_user_id)
        break

In [5]:
# Summary information

print("Max item user rated: %d"%max([len(i) for i in ratings]))
print("Min item user rated: %d"%min([len(i) for i in ratings]))
print("Mean item user rated: %d"%np.mean([len(i) for i in ratings]))


Max item user rated: 2314
Min item user rated: 20
Mean item user rated: 165


In [7]:
f = open("%s/ratings.txt"%DATA_DIR, "w")
for uid, u in enumerate(ratings):
    arr = np.array(u)
    arr = arr[np.argsort(arr[:, 2])]
    for i in arr:
        r = [str(j) for j in i]
        f.write("%d %s\n"%(uid, " ".join(r)))
f.close()

In [26]:
# write new id of items into file
tmp = [str(i) for i in item]
f = open("%s/item_id.txt"%DATA_DIR, "w")
f.write("\n".join(tmp))
f.close()

# Prepare Train, Test 

In [33]:
shuffle_id = np.random.permutation(n_user)
train_id = shuffle_id[604:]
test_id = shuffle_id[:604]

ftrain = open("%s/implicit/train.txt"%DATA_DIR, "w")
for idx in train_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftrain.write("%d %s\n"%(idx, " ".join(item)))
ftrain.close()

ftest = open("%s/implicit/test.txt"%DATA_DIR, "w")
for idx in test_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftest.write("%d %s\n"%(idx, " ".join(item)))
ftest.close()
    
    
                                          

## Create item description for cf-vae

In [9]:
# read movie description
des = list(open("data/ml-1m/movie_description.txt"))
des = [d.strip() for d in des]

# item id
item = sorted(data.itemID.unique())
movie_list = list(movies_info[0])

# matching
text = []
for i in item:
    idx = movie_list.index(i)
    text.append(des[idx*2+1])
text[:2]

['A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips.',
 'After being trapped in a jungle board game for 26 years, a Man-Child wins his release from the game. But, no sooner has he arrived that he is forced to play again, and this time sets the creatures of the jungle loose on th

In [12]:
# write file
# f = open("data/ml-1m/description_fix.txt", "w")
# f.write("\n".join(text))
# f.close()

# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)
save_npz("data/ml-1m/item.npz", X)

# Domain-to-domain

In [14]:
movies = pd.read_table("%s/movies.dat"%DATA_DIR, delimiter="::", header=None)
movies.rename(index=str, columns={0:'moviesID', 1:'title', 2:'genres'}, inplace=True)
movies.head()

/Users/linh/PhD/code/.env3/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,moviesID,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
movie_ids = list(open("../data/ml-1m/item_id.txt"))
movie_ids = [int(s.strip()) for s in movie_ids]
movies = movies[movies.moviesID.isin(movie_ids)]

In [25]:
movies = movies.values.tolist()
movies[-10:]

[[3943, 'Bamboozled (2000)', 'Comedy'],
 [3944, 'Bootmen (2000)', 'Comedy|Drama'],
 [3945, 'Digimon: The Movie (2000)', "Adventure|Animation|Children's"],
 [3946, 'Get Carter (2000)', 'Action|Drama|Thriller'],
 [3947, 'Get Carter (1971)', 'Thriller'],
 [3948, 'Meet the Parents (2000)', 'Comedy'],
 [3949, 'Requiem for a Dream (2000)', 'Drama'],
 [3950, 'Tigerland (2000)', 'Drama'],
 [3951, 'Two Family House (2000)', 'Drama'],
 [3952, 'Contender, The (2000)', 'Drama|Thriller']]

In [28]:
u = ratings[0]
arr = np.array(u)
arr = arr[np.argsort(arr[:, 2])]
for i in range(len(arr)):
    print(movies[arr[i, 0]], arr[i])

[3186, 'Girl, Interrupted (1999)', 'Drama'] [     2969         4 978300019]
[1721, 'Titanic (1997)', 'Drama|Romance'] [     1574         4 978300055]
[1022, 'Cinderella (1950)', "Animation|Children's|Musical"] [      957         5 978300055]
[1270, 'Back to the Future (1985)', 'Comedy|Sci-Fi'] [     1178         5 978300055]
[2340, 'Meet Joe Black (1998)', 'Romance'] [     2147         3 978300103]
[1836, 'Last Days of Disco, The (1998)', 'Drama'] [     1658         5 978300172]
[3408, 'Erin Brockovich (2000)', 'Drama'] [     3177         4 978300275]
[1207, 'To Kill a Mockingbird (1962)', 'Drama'] [     1117         4 978300719]
[2804, 'Christmas Story, A (1983)', 'Comedy|Drama'] [     2599         5 978300719]
[720, 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Animation'] [      689         3 978300760]
[1193, "One Flew Over the Cuckoo's Nest (1975)", 'Drama'] [     1104         5 978300760]
[260, 'Star Wars: Episode IV - A New Hope (1977)', 'Action|Adventure|Fantasy|Sc